# 赛题理解
赛题理解是切入一道赛题的基础，会影响后续特征工程和模型构建等各种工作，也影响着后续发展工作的方向，正确了解赛题背后的思想以及赛题业务逻辑的清晰，有利于花费更少时间构建更为有效的特征模型， 在各种比赛中， 赛题理解都是极其重要且必须走好的第一步， 今天我们就从赛题的理解出发， 首先了解一下这次赛题的概况和数据，从中分析赛题以及大致的处理方式， 其次我们了解模型评测的指标，最后对赛题的理解整理一些经验。

此次比赛是[新闻推荐场景下的用户行为预测挑战赛](https://tianchi.aliyun.com/competition/entrance/532063/introduction)， 该赛题是以新闻APP中的新闻推荐为背景， 目的是**要求我们根据用户历史浏览点击新闻文章的数据信息预测用户未来的点击行为， 即用户的最后一次点击的新闻文章**， 这道赛题的设计初衷是引导大家了解推荐系统中的一些业务背景， 解决实际问题。 

## 数据概况
该数据来自某新闻APP平台的用户交互数据，包括30万用户，近300万次点击，共36万多篇不同的新闻文章，同时每篇新闻文章有对应的embedding向量表示。为了保证比赛的公平性，从中抽取20万用户的点击日志数据作为训练集，5万用户的点击日志数据作为测试集A，5万用户的点击日志数据作为测试集B。具体数据表和参数， 大家可以参考赛题说明。下面说一下拿到这样的数据如何进行理解， 来有效的开展下一步的工作。

## 评价方式理解
理解评价方式， 我们需要结合着最后的提交文件来看， 根据sample.submit.csv， 我们最后提交的格式是针对每个用户， 我们都会给出五篇文章的推荐结果，按照点击概率从前往后排序。 而真实的每个用户最后一次点击的文章只会有一篇的真实答案， 所以我们就看我们推荐的这五篇里面是否有命中真实答案的。比如对于user1来说， 我们的提交会是：user1, article1, article2, article3, article4, article5$。

评价指标的公式如下：
$$
score(\text{user}) = \sum_{k=1}^5 \frac{s(\text{user}, k)}{k}
$$

假如article1就是真实的用户点击文章，也就是article1命中， 则$s(\text{user1},1)=1, s(\text{user1},2-4)$都是0， 如果article2是用户点击的文章， 则$s(\text{user},2)=1/2,s(\text{user},1,3,4,5)$都是0。也就是$score(\text{user})$命中第几条的倒数。如果都没中， 则$score(\text{user1})=0$。 这个是合理的， 因为我们希望的就是命中的结果尽量靠前， 而此时分数正好比较高。

## 问题分析
根据赛题简介，我们首先要明确我们此次比赛的目标： 根据用户历史浏览点击新闻的数据信息预测用户最后一次点击的新闻文章。从这个目标上看， 会发现此次比赛和我们之前遇到的普通的结构化比赛不太一样， 主要有两点：
- 首先是目标上， 要预测最后一次点击的新闻文章，也就是我们给用户推荐的是新闻文章， 并不是像之前那种预测一个数或者预测数据哪一类那样的问题
- 数据上， 通过给出的数据我们会发现， 这种数据也不是我们之前遇到的那种特征+标签的数据，而是基于了真实的业务场景， 拿到的用户的点击日志

所以拿到这个题目，我们的思考方向就是结合我们的目标，**把该预测问题转成一个监督学习的问题(特征+标签)，然后我们才能进行ML，DL等建模预测**。那么我们自然而然的就应该在心里会有这么几个问题：如何转成一个监督学习问题呢？ 转成一个什么样的监督学习问题呢？ 我们能利用的特征又有哪些呢？ 又有哪些模型可以尝试呢？ 此次面对数万级别的文章推荐，我们又有哪些策略呢？ 

当然这些问题不会在我们刚看到赛题之后就一下出来答案， 但是只要有了问题之后， 我们就能想办法解决问题了， 比如上面的第二个问题，转成一个什么样的监督学习问题？  由于我们是预测用户最后一次点击的新闻文章，从36万篇文章中预测某一篇的话我们首先可能会想到这可能是一个多分类的问题(36万类里面选1)， 但是如此庞大的分类问题， 我们做起来可能比较困难， 那么能不能转化一下？ 既然是要预测最后一次点击的文章， 那么如果我们能预测出某个用户最后一次对于某一篇文章会进行点击的概率， 是不是就间接性的解决了这个问题呢？概率最大的那篇文章不就是用户最后一次可能点击的新闻文章吗？ 这样就把原问题变成了一个点击率预测的问题(用户, 文章) --> 点击的概率(软分类)， 而这个问题， 就是我们所熟悉的监督学习领域分类问题了， 这样我们后面建模的时候， 对于模型的选择就基本上有大致方向了，比如最简单的逻辑回归模型。

这样， 我们对于该赛题的解决方案应该有了一个大致的解决思路，要先转成一个分类问题来做， 而分类的标签就是用户是否会点击某篇文章，分类问题的特征中会有用户和文章，我们要训练一个分类模型， 对某用户最后一次点击某篇文章的概率进行预测。 那么又会有几个问题：如何转成监督学习问题？ 训练集和测试集怎么制作？ 我们又能利用哪些特征？ 我们又可以尝试哪些模型？ 面对36万篇文章， 20多万用户的推荐， 我们又有哪些策略来缩减问题的规模？如何进行最后的预测？